In [9]:
import numpy as np
import random
import csv
import time
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import matplotlib.image as mpimg
import time

In [10]:
def compute_distance(current_x, current_y, possible_x, possible_y, env):
    '''Compute distance'''
    
    # Calculate distances considering wrap-around 
    dx = min(abs(possible_x - current_x), env.i - abs(possible_x - current_x))
    dy = min(abs(possible_y - current_y), env.j - abs(possible_y - current_y))
    
    # Compute the shortest distance 
    distance = np.sqrt(dx**2 + dy**2)
    return distance

def compute_attraction_repulsion(current_x, current_y, possible_x, possible_y, env):
    '''Compute attraction/repulsion based on the 
    inverse of the shortest distance in convex space.'''
      
    distance = compute_distance(current_x, current_y, possible_x, possible_y, env)
    
    if distance == 0:
        return 1e9  # Large number to avoid division by zero
    return 1 / (distance**2)


def compute_next_step(current_x, current_y, dx, dy, x_goal, y_goal, speed, env, attraction = True):
    '''Compute necessary speed based on nearest goal and 
    compute next step based on direction given in input, 
    return also the normalized direction'''
    
    # Initialize the best step size and the minimal distance to the goal
    best_step_size = 0
    min_distance = 1e9
    max_distance = -1
    best_new_x, best_new_y = 0, 0
    
    # Try different step sizes from 1 to max speed
    for step_size in range(1, speed + 1):
        
        # Normalize the direction
        normalized_dx = dx // abs(dx) if dx != 0 else 0
        normalized_dy = dy // abs(dy) if dy != 0 else 0
        
        # Scale the direction by the current step size
        scaled_dx = normalized_dx * step_size
        scaled_dy = normalized_dy * step_size
        
        # Compute the new position based on the current step size
        if attraction:
            new_x = (current_x + scaled_dx) % env.i
            new_y = (current_y + scaled_dy) % env.j
            
            
            distance_to_goal = compute_distance(new_x, new_y, x_goal, y_goal, env)
            
            
            if distance_to_goal < min_distance:
                min_distance = distance_to_goal
                best_step_size = step_size
                best_new_x, best_new_y = new_x, new_y
        else:
            new_x = (current_x - scaled_dx) % env.i
            new_y = (current_y - scaled_dy) % env.j
        
            
            distance_to_goal = compute_distance(new_y, new_y, x_goal, y_goal, env)
            
            # Update the best step if this step reduces the distance more
            if distance_to_goal > max_distance:
                max_distance = distance_to_goal
                best_step_size = step_size
                best_new_x, best_new_y = new_x, new_y
    
    
    return best_new_x, best_new_y, normalized_dx, normalized_dy



In [11]:
def display_simulation_step(env, step, agent = None, run = 0, terminated= False):
    """Display the simulation step-by-step with images for mouse, cheese, and cat."""
    visualization_time = 0.6
    fig_width = 9
    fig_height = fig_width * (env.i / env.j) if env.i > env.j else fig_width * (env.j / env.i)
    fig, ax = plt.subplots(figsize=(fig_width, fig_height))

    
    fig.patch.set_facecolor('#f4f7fa')  
    ax.set_facecolor('#ffffff') 

    
    for x in range(env.i):
        for y in range(env.j):
            color = '#e3f2fd' if (x + y) % 2 == 0 else '#bbdefb'  
            ax.add_patch(plt.Rectangle((y - 0.5, env.i - x - 1 - 0.5), 1, 1, color=color, zorder=0))

    # Load the images for the agents and cheese
    cat_img = mpimg.imread("./images/cat.png")
    mouse_img = mpimg.imread("./images/mouse.png")
    cheese_img = mpimg.imread("./images/cheese.png")

    
    for x in range(env.i):
        for y in range(env.j):
            if env.grid[x, y] == 2:  # Cat
                ax.imshow(cat_img, extent=[y - 0.5, y + 0.5, env.i - x - 1 - 0.5, env.i - x - 1 + 0.5], 
                          zorder=1, aspect="auto")
            elif env.grid[x, y] == 1:  # Mouse
                ax.imshow(mouse_img, extent=[y - 0.5, y + 0.5, env.i - x - 1 - 0.5, env.i - x - 1 + 0.5], 
                          zorder=1, aspect="auto")
            elif env.grid[x, y] == 3:  # Cheese
                ax.imshow(cheese_img, extent=[y - 0.5, y + 0.5, env.i - x - 1 - 0.5, env.i - x - 1 + 0.5], 
                          zorder=1, aspect="auto")

   
    ax.set_xlim(-0.5, env.j - 0.5)
    ax.set_ylim(-0.5, env.i - 0.5)

    
    ax.set_aspect('equal')

    
    if agent and not terminated:
        ax.set_title(f"Run {run } | Step {step} | Agent {agent.agent_type} ID {agent.agent_id}", fontsize=18, fontweight='bold', 
                    color="#0288d1", ha='center', family='Arial')
        ax.text(0.5, -0.12, f"\n Status: {'DEAD' if agent.dead else 'ALIVE'}\nPosition: ({agent.x}, {agent.y}) | Max Speed: {agent.speed} | Eaten: {agent.eaten}\nPath: {len(agent.path)} steps ",
                transform=ax.transAxes, fontsize=13, ha='center', color="#01579b", family='Arial')
        #fig.savefig(f'fig{run}_{step}_{agent.agent_type}_{agent.agent_id}.png')
    elif terminated and agent is None:
        ax.set_title(f"Step {step} | The simulation is terminated at Run {run} and Step {step}!\n No more Mice left!", fontsize=18, fontweight='bold', 
                    color="#0288d1", ha='center', family='Arial')
        #fig.savefig(f'fig{run}_{step}_terminated.png')
    else: 
        ax.set_title(f"Step {step} | Initial State", fontsize=18, fontweight='bold', 
                    color="#0288d1", ha='center', family='Arial')
        #fig.savefig(f'fig{run}_{step}_initial.png')

    
    ax.grid(False)  
    ax.set_xticks(np.arange(-0.5, env.j, 1))
    ax.set_yticks(np.arange(-0.5, env.i, 1))
    ax.set_xticklabels([])
    ax.set_yticklabels([])

    clear_output(wait=True)
    display(fig)
    plt.close(fig)
    time.sleep(visualization_time)
    


In [12]:
def save_data(agents, filename, run):
    '''Save simulation data in the given csv file'''
    
    with open(filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for agent in agents:
            for step in agent.path:
                writer.writerow([run, agent.agent_id, agent.agent_type, step[0], step[1], step[2], step[3], step[4]])
       

In [13]:
class Agent:
    def __init__(self, x, y, agent_type, agent_id, speed):
        self.x = x 
        self.y = y
        self.agent_type = agent_type
        self.agent_id = agent_id 
        self.path = [(0, self.x, self.y, '0', 0)] # (path length, x, y, last movement direction, eaten)
        self.eaten = 0
        self.directions = {
            (1, 0): 'N',    # North
            (-1, 0): 'S',     # South
            (0, 1): 'W',    # West
            (0, -1): 'E',     # East
            (1, 1): 'NW',  # Northwest
            (1, -1): 'NE',   # Northeast
            (-1, 1): 'SW',   # Southwest
            (-1, -1): 'SE',    # Southeast      
        }
        self.dead = False
        self.speed= speed

        
    def move(self, env):
        '''
        It verifies if there are any attractive/repulsive cells, based on the agent type,
        in vision range. If are present calculates attractiveness or repulsion and saves 
        direction and corrisponding value. 
        After the iteration (if any is found) checks from the higher atraction/repulsion 
        direction. If the cell where the agent would move is free calls result(...), 
        otherwise choose the following higher attraction/repulsion. 
        If nothig is available or there isn't any interresting cell in vision range it moves
        randomly where is possible.
        '''
        
        vision_range = abs(env.vision_range)
        next_dx, next_dy = (0,0)
        

        if self.agent_type == "cat":
            possible_cat_directions = []

            # checks cell in vision range
            for dx in range(-vision_range, vision_range + 1):
                for dy in range(-vision_range, vision_range + 1):
                    if dx == 0 and dy == 0:
                        continue
                    new_x, new_y = (self.x + dx) % env.i, (self.y + dy) % env.j
                    if env.grid[new_x, new_y] == 1:
                        attraction = compute_attraction_repulsion(self.x, self.y, new_x, new_y, env)
                        possible_cat_directions.append([(dx, dy), attraction, (new_x, new_y)])
                        
                  
            if possible_cat_directions:
                possible_cat_directions.sort(key=lambda x: x[1], reverse=True)

                for direction in possible_cat_directions:
                    next_dx, next_dy = direction[0]
                    x_goal, y_goal = direction[2]
                        
                    new_x, new_y, next_dx, next_dy = compute_next_step(self.x, self.y, next_dx, next_dy, x_goal, y_goal, self.speed, env)

                    if env.grid[new_x, new_y] != 3 and env.grid[new_x, new_y] != 2: # if the cell is not occupied by cheese or another Cat, the Cat can move
                        self.results(new_x, new_y, next_dx, next_dy, env)
                        break
            else: # if there isn't any attractive cell, move randomly
                dx, dy = random.choice(list(self.directions.keys()))
                new_x = (self.x + dx) % env.i
                new_y = (self.y + dy) % env.j
                while env.grid[new_x, new_y] != 0:
                    dx, dy = random.choice(list(self.directions.keys()))
                    new_x = (self.x + dx) % env.i
                    new_y = (self.y + dy) % env.j
                        
                self.results(new_x, new_y, dx, dy,env)

        elif self.agent_type == "mouse":
            if not self.dead:
                possible_directions_attraction, possible_directions_repulsion = [], []
                move_made = False

                for dx in range(-vision_range, vision_range + 1):
                    for dy in range(-vision_range, vision_range + 1):
                        if dx == 0 and dy == 0:
                            continue
                        new_x, new_y = (self.x + dx) % env.i, (self.y + dy) % env.j
                        if env.grid[new_x, new_y] == 3:
                            attraction = compute_attraction_repulsion(self.x, self.y, new_x, new_y, env)
                            possible_directions_attraction.append([(dx, dy), attraction, (new_x, new_y)])
                        elif env.grid[new_x, new_y] == 2:
                            repulsion = env.beta * compute_attraction_repulsion(self.x, self.y, new_x, new_y, env)
                            possible_directions_repulsion.append([(dx, dy), repulsion, (new_x, new_y)])

                # if there are Mouse and chesse in range mest evaluate the best direction
                if possible_directions_attraction and possible_directions_repulsion:
                    
                    possible_directions_attraction.sort(key=lambda x: x[1], reverse=True)
                
                    possible_directions_repulsion.sort(key=lambda x: x[1], reverse=True)
                    
                    
                    for direction_away_cat in possible_directions_repulsion:
                        for direction_cheese in possible_directions_attraction:
                            if direction_away_cat[1] >= direction_cheese[1]: # if cat is the nearest, go away from it
                                next_dx, next_dy = direction_away_cat[0]
                                x_goal, y_goal = direction_away_cat[2]
                                
                                new_x, new_y, next_dx, next_dy = compute_next_step(self.x, self.y, next_dx, next_dy, x_goal, y_goal, self.speed, env, attraction= False)
                                if env.grid[new_x, new_y] != 1 and env.grid[new_x, new_y] != 2: # if the cell is not occupied by a Cat or another Mouse, tha Mouse can move
                                    self.results(new_x, new_y, next_dx, next_dy, env)
                                    move_made = True
                                    break
                            else:
                                next_dx, next_dy = direction_cheese[0]
                                x_goal, y_goal = direction_cheese[2]
                                
                                new_x, new_y, next_dx, next_dy = compute_next_step(self.x, self.y, next_dx, next_dy, x_goal, y_goal, self.speed, env)
                                if env.grid[new_x, new_y] != 1 and env.grid[new_x, new_y] != 2: # if the cell is not occupied by a Cat or another Mouse, tha Mouse can move
                                    self.results(new_x, new_y, next_dx, next_dy, env)
                                    move_made = True
                                    break
                        if move_made:
                            break
                
                # if only Cats are in range           
                elif possible_directions_repulsion and not possible_directions_attraction: 
                    possible_directions_repulsion.sort(key=lambda x: x[1], reverse=True)
                    for direction_away_cat in possible_directions_repulsion:
                        next_dx, next_dy = direction_away_cat[0]
                        x_goal, y_goal = direction_away_cat[2]
                                
                        new_x, new_y, next_dx, next_dy = compute_next_step(self.x, self.y, next_dx, next_dy, x_goal, y_goal, self.speed, env, attraction= False)
                        if env.grid[new_x, new_y] != 1 and env.grid[new_x, new_y] != 2: # if the cell is not occupied by a Cat or another Mouse, tha Mouse can move
                            self.results(new_x, new_y, next_dx, next_dy, env)
                            break
                
                # if only cheese is in range        
                elif possible_directions_attraction and not possible_directions_repulsion:
                    possible_directions_attraction.sort(key=lambda x: x[1], reverse=True)
                    for direction_cheese in possible_directions_attraction:
                        next_dx, next_dy = direction_cheese[0]
                        x_goal, y_goal = direction_cheese[2]
                                
                        new_x, new_y, next_dx, next_dy = compute_next_step(self.x, self.y, next_dx, next_dy, x_goal, y_goal, self.speed, env)
                        if env.grid[new_x, new_y] != 1 and env.grid[new_x, new_y] != 2: # if the cell is not occupied by a Cat or another Mouse, tha Mouse can move
                            self.results(new_x, new_y, next_dx, next_dy, env)
                            break
                
                # if there is no cell with Cat or cheese in range, choose random direction        
                else: 
                    dx, dy = random.choice(list(self.directions.keys()))
                    new_x = (self.x + dx) % env.i
                    new_y = (self.y + dy) % env.j
                    while env.grid[new_x, new_y] != 0:
                        dx, dy = random.choice(list(self.directions.keys()))
                        new_x = (self.x + dx) % env.i
                        new_y = (self.y + dy) % env.j
                        
                    self.results(new_x, new_y, dx, dy,env)
                
                         
            
        
        

    def results(self, new_x, new_y, next_dx, next_dy,env):
        '''
        First of all checks if the agent is alive (the Mouse can die).
        Then the current cell is freed.
        If it's a Cat check if the next cell contains a Mouse, in this 
        case the Cat eats it and updates its status, otherwise just 
        updates its position and the envirement.
        If it's a Mouse then checks for cheese and if present eats it 
        updating its status. Otherwise, just updates its position and
        the envirement.   
        The agent path is finally updated.    
        
        '''
        
        if not self.dead:
            if self.agent_type == 'cat':
                env.grid[self.x, self.y] = 0 # the Cat frees its current location
                if env.grid[new_x, new_y] == 1: # if there is a Mouse, eat it
                    self.eaten += 1
                    
                    # the Mouse eaten must be removed from envirement
                    env.kill_mouse(new_x, new_y)
                
                # update grid and Cat position
                env.grid[new_x, new_y] = 2
                self.x = new_x
                self.y = new_y
                
            elif self.agent_type == 'mouse':
                env.grid[self.x, self.y] = 0 # the Mouse frees its current location
                if env.grid[new_x, new_y] == 3: # if there is cheese, eat it
                    self.eaten += 1
                    env.grid[new_x, new_y] = 0
                    self.relocate(env) # the Mouse must be relocated
                else:
                    env.grid[new_x, new_y] = 1
                    self.x = new_x
                    self.y = new_y
                    
                
            self.path.append((len(self.path), round(self.x), round(self.y), self.directions[next_dx, next_dy], self.eaten))
        
            
                
    def relocate(self, env):
        '''
        When a mouse eats cheese, it must be relocated in the 
        envirement.
        '''
        
        if self.agent_type == 'mouse':
            x, y = np.random.randint(0, env.i), np.random.randint(0, env.j)
            while env.grid[x, y] != 0:
                x, y = np.random.randint(0, env.i), np.random.randint(0, env.j)
            env.grid[x, y] = 1
            self.x = x
            self.y = y  
            
                        
            
    
                                

In [14]:
class Envirement:
    
    used_ids = set()
    
    def __init__(self, i: int = 10, j: int = 10 , C: int = 3, H: int = 5, M: int = 4, random_seed: int = 42, vision_range: int = 4, beta: float = 1.0, k: int = 2, total_run: int = 2, uniform_speed: bool = True):
        if not isinstance(i, int) or i <= 0:
            raise ValueError("i - number of row - must be a positive integer.")
        if not isinstance(j, int) or j <= 0:
            raise ValueError("j - number of columns - must be a positive integer.")
        if not isinstance(C, int) or C <= 0:
            raise ValueError("C - number of Cats - must be a positive integer.")
        if not isinstance(H, int) or H <= 0:
            raise ValueError("H - number of cheese - must be a positive integer.")
        if not isinstance(M, int) or M <= 0:
            raise ValueError("M - number of Mice - must be a positive integer.")
        if not isinstance(random_seed, int) or random_seed <= 0:
            raise ValueError("Random seed must be a positive integer.")
        if not isinstance(vision_range, int) or vision_range <= 0 or vision_range > min(i,j):
            raise ValueError("Vision range must be a positive integer and inside grid limit.")
        if not isinstance(beta, float) or beta < 1:
            raise ValueError("Beta must be a positive float, greater than 1.")
        if not isinstance(k, int) or k <= 0:
            raise ValueError("k - number of steps - must be a positive integer.")
        if not isinstance(total_run, int) or total_run <= 0:
            raise ValueError("total_run - number of iterations - must be a positive integer.")
        if not isinstance(uniform_speed, bool):
            raise ValueError("uniform_speed - agent's speeds - must be a boolean.")
        
        self.i = i
        self.j = j
        self.C = C
        self.H = H
        self.M = M
        self.random_seed = random_seed
        self.vision_range = vision_range
        self.beta = beta
        self.k = k
        self.total_run = total_run
        self.cats = []
        self.mice = []
        self.agents=[]
        self.grid = np.zeros((self.i, self.j))
        self.execution = False
        self.mice_left = M
        self.uniform_speed = uniform_speed
        
        
        
        random.seed(self.random_seed)
        np.random.seed(self.random_seed)
        
        
        # Cheese position
        for _ in range(self.H):
            x, y = np.random.randint(0, self.i), np.random.randint(0, self.j)
            while self.grid[x, y] != 0:
                x, y = np.random.randint(0, self.i), np.random.randint(0, self.j)
            self.grid[x, y] = 3

        # Mouse Position
        for i in range(self.M):
            x, y = np.random.randint(0, self.i), np.random.randint(0, self.j)
            while self.grid[x, y] != 0:
                x, y = np.random.randint(0, self.i), np.random.randint(0, self.j)
            self.grid[x, y] = 1
            self.mice.append(Agent(
                x, 
                y, 
                "mouse", 
                self.generate_unique_id(), 
                speed=1 if self.uniform_speed else np.random.randint(1, self.vision_range)
            ))

        # Cats position
        for i in range(self.C):
            x, y = np.random.randint(0, self.i), np.random.randint(0, self.j)
            while self.grid[x, y] != 0:
                x, y = np.random.randint(0, self.i), np.random.randint(0, self.j)
            self.grid[x, y] = 2
            self.cats.append(Agent(
                x, 
                y, 
                "cat", 
                self.generate_unique_id(), 
                speed=1 if self.uniform_speed else np.random.randint(1, self.vision_range+1)
            ))
            
        self.agents = self.alternate_agents()

    
    def generate_unique_id(self):
        ''' 
        Generates ramdom ID
        '''
        max_id = self.M + self.C  
        while True:
            new_id = random.randint(0, max_id - 1) 
            if new_id not in self.used_ids:
                self.used_ids.add(new_id) 
                return new_id
            
    
    def alternate_agents(self):
        ''' 
        After agents are created and positioned it generates a list
        alternating Mice and Cats.
        '''
        
        alternating_array = []
        min_len = min(len(self.cats), len(self.mice))
        
        for i in range(min_len):
            alternating_array.append(self.mice[i])
            alternating_array.append(self.cats[i])
        
        
        if len(self.cats) > len(self.mice):
            alternating_array.extend(self.cats[min_len:])
        else:
            alternating_array.extend(self.mice[min_len:])
        
        return alternating_array
    
    
    
    def kill_mouse(self, x, y):
        ''' 
        When a Mouse it's killed by a Cat it must be removed 
        from envirement and its status must be correctly updated.
        '''      
          
        for agent in self.mice:
            if agent.agent_type == "mouse" and agent.x == x and agent.y == y:
                 
                agent.dead = True
                agent.x = -1
                agent.y = -1
                self.grid[x, y] = 0
                self.mice_left -=1
                break  
                
        return
    
    
    def display_final_statistics(self, verbose = False, path = False):
        ''' 
        Final statistic are displayd.
        If verbose = True ID, death status and eaten are shown for each 
        agent. 
        If path = True the travelled path is shown for each agent.
        '''
        
        if self.execution:
            mice_alive = 0
            cheese_available = 0
            print('\n\n------------FINAL STATISTICS------------')
            for i in range(self.i):
                for j in range(self.j):
                    if self.grid[i,j] == 1:
                        mice_alive += 1
                    if self.grid[i, j] == 3:
                        cheese_available +=1
                        
            if mice_alive == 0:
                print('All Mice are DEAD!\nCATS HAVE WON!')
                print("""
                /\_/\\  
                ( o.o ) 
                > ^ <
                """)
            if mice_alive != 0 and cheese_available == 0:
                print(f'{mice_alive} Mice are ALIVE and have eaten ALL CHEESE!\nMICE HAVE WON!')
                print("""
                (\\(\\  
                ( -.-)  
                o_(")(")   ____
                           \\  /
                            \\/
                """)
            elif mice_alive and cheese_available != 0:
                print(f'{mice_alive} Mice are ALIVE but there is some cheese left ({cheese_available})!\nTIE!')
                print("""
                (\\(\\        /\_/\\  
                ( -.-)      ( o.o ) 
                o_(")(")     > ^ <
                """)

            if verbose:
                print('\n\n~~~~~~~~~~~More details~~~~~~~~~~~')
                print('\n\n----------MICE---------')
                for mouse in self.mice:
                    print(f'Mouse with ID {mouse.agent_id} death status is: {mouse.dead} and has eaten {mouse.eaten} pieces of cheese.\n It traveled for {len(mouse.path)} cells.')
                    if path:
                        print(f'Mouse speed: {mouse.speed}\nMouse Path (path length, x, y, last movement direction, eaten): \n{mouse.path}\n\n')
                    
                print('\n\n----------CATS---------')
                for cat in self.cats:
                    print(f'Cat with ID {cat.agent_id} has eaten {cat.eaten} mice.\n It traveled for {len(cat.path)} cells.')
                    if path:
                        print(f'Cat speed: {cat.speed}\nCat Path (path length, x, y, last movement direction, eaten): \n{cat.path}\n\n')   
                    
       
    def run_simulation(self, show_steps, run = 0):
        ''' 
        The simulation is run for k steps, where each one is 
        displayed for visualization_time time.
        If there aren't any more mice, the simulation stops.
        '''
        if show_steps:
            display_simulation_step(self, 0, None, run)
        
        for step in range(self.k):
            if self.mice_left == 0:
                if show_steps:
                    display_simulation_step(self, step, None, run, True)
                break

            # Move all the mice first
            for mouse in self.mice:
                if self.mice_left == 0:  # Check again in case all Mice are gone mid-step
                    if show_steps:
                        display_simulation_step(self, step, None, run, True)
                    break
                mouse.move(self)
                if show_steps:
                    display_simulation_step(self, step, mouse, run)
            
            # Then all Cats
            for cat in self.cats:
                if self.mice_left == 0:  # Check again in case all Mice are gone mid-step
                    if show_steps:
                        display_simulation_step(self, step, None, run, True)
                    break
                cat.move(self)
                if show_steps: 
                    display_simulation_step(self, step, cat, run)
                
            
        
    def run(self, show_steps):
        '''
        The csv file name is created based on time and date and 
        inizialized with headers. 
        The simulation is run total_run times with k steps. Each
        step is shown for visualization_time time.
        '''
        
        self.execution = True
        current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")
        title_with_time = f"simulation_data_{current_time}.csv"
        headers = [
            ["Run", "Agent_id", "Agent_kind", "Path_length", "x_position", "y_position", "last_movement_direction", "n_eaten"],  # Headers for the csv file
        ]
        
        with open(title_with_time, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(headers)
            
        for run in range(self.total_run):
            if self.mice_left>0:  
                self.run_simulation(show_steps, run)
            else: 
                break
                
            save_data(self.agents, title_with_time, run)
        
        print(f"Execution terminated.\nData have been successfully collected in {title_with_time}.")   
        return title_with_time
        
              

<>:166: SyntaxWarning: invalid escape sequence '\_'
<>:182: SyntaxWarning: invalid escape sequence '\_'
<>:166: SyntaxWarning: invalid escape sequence '\_'
<>:182: SyntaxWarning: invalid escape sequence '\_'
C:\Users\sarag\AppData\Local\Temp\ipykernel_18712\1105390250.py:166: SyntaxWarning: invalid escape sequence '\_'
  print("""
C:\Users\sarag\AppData\Local\Temp\ipykernel_18712\1105390250.py:182: SyntaxWarning: invalid escape sequence '\_'
  print("""


#### Big Simulation

In [15]:
'''RANDOM_SEED = 47
env = Envirement(i = 1200, j= 1500, C = 800, H = 600 , M = 1000, random_seed = RANDOM_SEED, vision_range= 70, beta = 3.2, k = 7, total_run = 3, uniform_speed = False)
    
file_name = env.run(False)
env.display_final_statistics(verbose = True, path = True)
'''

'RANDOM_SEED = 47\nenv = Envirement(i = 1200, j= 1500, C = 800, H = 600 , M = 1000, random_seed = RANDOM_SEED, vision_range= 70, beta = 3.2, k = 7, total_run = 3, uniform_speed = False)\n    \nfile_name = env.run(False)\nenv.display_final_statistics(verbose = True, path = True)\n'

#### Standard Simulation 

In [16]:
'''RANDOM_SEED = 47
env = Envirement(i = 8, j= 8, C = 3, H = 4, M = 3, random_seed = RANDOM_SEED, vision_range= 4, beta = 3.2, k = 3, total_run = 2, uniform_speed = False)

file_name = env.run(False)
env.display_final_statistics(verbose = True, path = True)'''

'RANDOM_SEED = 47\nenv = Envirement(i = 8, j= 8, C = 3, H = 4, M = 3, random_seed = RANDOM_SEED, vision_range= 4, beta = 3.2, k = 3, total_run = 2, uniform_speed = False)\n\nfile_name = env.run(False)\nenv.display_final_statistics(verbose = True, path = True)'